## Semantic Chunking and RAG Fusion

- This Notebook does semantic chunking of csv files otained from notebooks/summarize_image.ipynb, it creates chunks in such a way that the related textual content will be present in the same chunk, which are inserted into QDrant a collecction "owners_manual_chunks". 

- RAG Fusion is a process where, multiple queries will be created similar to the users query with the help of a LLM. With the help of reciprocal rank fusion, the documents are ranked and provided to the chat completion LLM as context.

In [1]:
%%capture
!pip install langchain
!pip install langchain_openai langchain_experimental
!pip install langchainhub
!pip install langchain_together
!pip -q install --upgrade together
!pip install fastembed
!pip install qdrant-client
!pip install sentence_transformers
!pip install langchain-cohere
!pip install einops
!pip install PyPDF2
!pip install pypdf
!pip install -U langchain-qdrant

In [32]:
import os
os.environ["TOGETHER_API_KEY"] = "148521c4088ad416dced465cc144671626b00c860af4e6ebc855953567087d8a"
os.environ["COHERE_API_KEY"] = 'xxe3X6u8vcTFJgJ8Pc7CfLezwpQiATQcUB56VIUp'

TOP_K = 5
MAX_DOCS_FOR_CONTEXT = 5

In [60]:
from operator import itemgetter
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
from qdrant_client.http import models
from langchain_core.documents.base import Document
from langchain.load import dumps, loads
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.retrievers import BaseRetriever
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain_cohere.llms import Cohere
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from sentence_transformers.util import cos_sim
import PyPDF2
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_qdrant import Qdrant

In [ ]:
embedding_model = CohereEmbeddings(model = "embed-english-v3.0")
semantic_chunker_embed_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

In [45]:
qdrant_client = QdrantClient(
    "https://8803fa99-7551-4f88-84c3-e134c9bed5de.us-east4-0.gcp.cloud.qdrant.io:6333",
    prefer_grpc=True,
    api_key="EFeN_UhdmAlDNYZHqJBUbZ88Nt7N0MkmvWLgM5Hs4ogNvExLMwNwdQ",
)

QDRANT_URL = "https://8803fa99-7551-4f88-84c3-e134c9bed5de.us-east4-0.gcp.cloud.qdrant.io:6333"
QDRANT_API_KEY = "EFeN_UhdmAlDNYZHqJBUbZ88Nt7N0MkmvWLgM5Hs4ogNvExLMwNwdQ"

In [ ]:
def create_QDrant_collection():
		embeddings = CohereEmbeddings(model = "embed-english-v3.0")
		qdrant_client.recreate_collection(
		collection_name="owners_manual",
		vectors_config = models.VectorParams(size=1024, distance=models.Distance.COSINE),
	)

create_QDrant_collection()

In [17]:
def upload_chunks_to_QDrant(documents):
  records_to_upload = []
  for idx, chunk in enumerate(documents):
      content = chunk.page_content
      vector = embedding_model.encode(content).tolist()

      record = models.PointStruct(
          id=idx,
          vector=vector,
          payload={"page_content": content}
      )
      records_to_upload.append(record)

  qdrant_client.upload_points(
      collection_name="owners_manual",
      points=records_to_upload
  )

In [39]:
def upload_pdf_to_QDrant(pdf_path):
    semantic_chunker = SemanticChunker(
        semantic_chunker_embed_model, breakpoint_threshold_type="percentile"
    )

    pdf_file_path = pdf_path
    pdf_file = open(pdf_file_path, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    text = ''
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        documents = semantic_chunker.create_documents([text])
        upload_chunks_to_QDrant(documents)

    pdf_file.close()

In [ ]:
upload_pdf_to_QDrant("/content/text.pdf")

In [50]:
loader = PyPDFLoader("/content/text.pdf")
pages = loader.load_and_split()

qdrant = Qdrant.from_documents(
    pages,
    embedding_model,
    url=QDRANT_URL,
    prefer_grpc=True,
    api_key=QDRANT_API_KEY,
    collection_name="owners_manual",
)

In [57]:
template = """Please answer the [question] using only the following [information]. If there is no [information] available to answer the question, do not force an answer.

Information: {context}

Question: {question}
Final answer:"""

In [56]:
def reciprocal_rank_fusion(results: list[list], k=60):
    """Rerank docs (Reciprocal Rank Fusion)

    Args:
        results (list[list]): retrieved documents
        k (int, optional): parameter k for RRF. Defaults to 60.

    Returns:
        ranked_results: list of documents reranked by RRF
    """

    fused_scores = {}
    for docs in results:
        # Assumes the docs are returned in sorted order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # return only documents
    return [x[0] for x in reranked_results[:MAX_DOCS_FOR_CONTEXT]]

In [55]:
def query_generator(original_query: dict) -> list[str]:
    """Generate queries from original query

    Args:
        query (dict): original query

    Returns:
        list[str]: list of generated queries
    """

    # original query
    query = original_query.get("query")

    # prompt for query generator
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that generates multiple search queries based on a single input query."),
        ("user", "Generate multiple search queries related to:  {original_query}. When creating queries, please refine or add closely related contextual information, without significantly altering the original query's meaning"),
        ("user", "OUTPUT (3 queries):")
    ])

    # LLM model
    model = Cohere()

    # query generator chain
    query_generator_chain = (
        prompt | model | StrOutputParser() | (lambda x: x.split("\n"))
    )

    # gererate queries
    queries = query_generator_chain.invoke({"original_query": query})

    # add original query
    queries.insert(0, "0. " + query)

    return queries

In [54]:
def rrf_retriever(query: str) -> list[Document]:
    """RRF retriever

    Args:
        query (str): Query string

    Returns:
        list[Document]: retrieved documents
    """

    # Retriever
    embedding = CohereEmbeddings(model = "embed-english-v3.0")


    qdrant = Qdrant(
        client=qdrant_client,
        collection_name="owners_manual",
        embeddings=embedding,
    )

    retriever = qdrant.as_retriever()

    # RRF chain
    chain = (
        {"query": itemgetter("query")}
        | RunnableLambda(query_generator)
        | retriever.map()
        | reciprocal_rank_fusion
    )

    # invoke
    result = chain.invoke({"query": query})

    return result

In [53]:
def query(query: str, retriever: BaseRetriever):
    """
    Query with vectordb
    """

    # model
    model = Cohere()

    # prompt
    prompt = PromptTemplate(
        template=template,
        input_variables=["context", "question"],
    )

    # Query chain
    chain = (
        {
            "context": itemgetter("question") | retriever,
            "question": itemgetter("question")
        }
        | RunnablePassthrough.assign(
            context=itemgetter("context")
        )
        | {
            "response": prompt | model | StrOutputParser(),
            "context": itemgetter("context"),
        }
    )

    # execute chain
    result = chain.invoke({"question": query})

    return result

In [61]:
retriever = RunnableLambda(rrf_retriever)
result = query("Explain Hyundai Warranty Policy", retriever)
result['response']

' Hyundai provides a 3-year warranty from the date of sale, and the warranty is transferable to the subsequent owner for the remaining warranty period. The warranty policy offers comprehensive support to customers, including emergency roadside assistance, roadside repair, vehicle recovery, and jumping a dead battery. Customers are advised to visit a authorized Hyundai dealer workshop for assistance,  the full terms and conditions of which can be found online at the provided URL. \n\nIs there anything else you would like to know about Hyundai Warranty Policy? '